In [ ]:
Id=[]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/fakereal-logo-detection-dataset'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
Id[:5]

In [ ]:
train=pd.DataFrame()
train=train.assign(filename=Id[2:])
train.head()

In [ ]:
train['label']=train['filename']
train['label']=train['label'].str.replace('/kaggle/input/fakereal-logo-detection-dataset/','')
train.head()

In [ ]:
train['label'] = train['label'].str.split('/').str[0]
train.head()

In [ ]:
train['label'] = train['label'].replace({'output':'Genuine','genLogoOutput':'Fake'})
train.head()

In [ ]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

In [ ]:
from sklearn.model_selection import train_test_split
train,test= train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=train['label'])

In [ ]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

In [ ]:
train=train.sample(n=len(train))
train.reset_index(inplace=True,drop=True)
train.head()

In [ ]:
test=test.sample(n=len(test))
test.reset_index(inplace=True,drop=True)
test.head()

In [ ]:
train.shape,test.shape

In [ ]:
import os
import cv2
import multiprocessing
from functools import partial
import pandas as pd
import uuid

def _process_df_chunk(chunk, file_col, label_col, dest_folder):
    def _process_image(file_path, label):
        try:
            # Load the image
            img = cv2.imread(file_path)
            # Check if the image size is empty
            if img is None or img.size == 0:
                print(f"Error: Could not read file {file_path}")
                return
            # Resize to 512x512
            img = cv2.resize(img, (512, 512))
            # Save the image to the folder with label name
            folder_path = os.path.join(dest_folder, str(label))
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            file_name = os.path.basename(file_path)
            # Append a unique identifier to the file name to avoid overwriting
            name, extension = os.path.splitext(file_name)
            save_path = os.path.join(folder_path, f"{name}_{uuid.uuid4().hex}{extension}")
            cv2.imwrite(save_path, img)
        except Exception as e:
            print(f"Error: {e}")
            
    for _, row in chunk.iterrows():
        _process_image(row[file_col], row[label_col])

def resize_save_image(df, file_col, label_col, dest_folder):
    try:
        # Determine the number of processes to use
        num_processes = multiprocessing.cpu_count()

        # Split the DataFrame into chunks to process in parallel
        df_chunks = [df[i:i + num_processes] for i in range(0, len(df), num_processes)]

        # Create a partial function with fixed arguments
        func = partial(_process_df_chunk, file_col=file_col, label_col=label_col, dest_folder=dest_folder)

        # Run the function in parallel on the DataFrame chunks
        with multiprocessing.Pool(processes=num_processes) as pool:
            pool.map(func, df_chunks)
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
import os
newpath = r'./train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [ ]:
resize_save_image(train, file_col='filename', label_col='label' ,dest_folder='./train')

In [ ]:
newpath = r'./test' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [ ]:
resize_save_image(test, file_col='filename', label_col='label' ,dest_folder='./test')